In [1]:
import multiprocessing
from pathlib import Path

import pandas as pd
import numpy as np
from tqdm import tqdm
from joblib import Parallel, delayed

# Opérations de traitements


In [7]:
def clean_df(df:pd.DataFrame)->pd.DataFrame:
  df=df.drop(columns=['ancien_code_commune','ancien_nom_commune','ancien_id_parcelle'])
  # Elimination des locaux industriels et commerciaux
  mutation_commerciale=df[df.code_type_local==4].id_mutation
  df=df[~df.id_mutation.isin(mutation_commerciale)]
  coltofill=['valeur_fonciere','nombre_pieces_principales','surface_reelle_bati',
            'surface_terrain','lot1_surface_carrez','lot2_surface_carrez',
            'lot3_surface_carrez','lot4_surface_carrez','lot5_surface_carrez',
            'lot1_numero','lot2_numero','lot3_numero','lot4_numero','lot5_numero',
            'adresse_numero','code_postal','numero_volume','longitude','latitude']
  df[coltofill]=df[coltofill].fillna(0)
  df['code_type_local']=df['code_type_local'].fillna(3)
  df['type_local']=df['type_local'].fillna('Dépendance')

  coltofill=['adresse_nom_voie','adresse_code_voie','code_nature_culture',
            'adresse_suffixe','nature_culture','code_nature_culture_speciale',
            'nature_culture_speciale']
  df[coltofill]=df[coltofill].fillna('<EMPTY>')
  return df


# Fonctions de transformations des données en version linéarisée

In [6]:
alt_df_cols=[
             "id_mutation",
             'jour_mutation',
             'mois_mutation',
             'annee_mutation',
             "nature_mutation",
             "valeur_fonciere",
             "adresse_numero",
             "adresse_suffixe",
             'adresse_nom_voie',
             'adresse_code_voie',
             'code_postal',
             'nom_commune',
             'code_departement',
             'id_parcelle',
             'surface_carrez_total',
             'surface_reelle_bati_total',
             'surface_terrain_total',
             'nombre_lots',
             'nombre_maisons',
             'surface_carrez_maisons',
             'surface_reelle_bati_maisons',
             'surface_terrain_maisons',
             'nombre_appartements',
             'surface_carrez_appartements',
             'surface_reelle_bati_appartements',
             'surface_terrain_appartements',
             'nombre_dependances',
             'surface_carrez_dependances',
             'surface_reelle_bati_dependances',
             'surface_terrain_dependances',
             'nombre_pieces_principales',
             'nature_culture',
             'nature_culture_speciale',
             'longitude',
             'latitude',
]

def reduce_get_id(df:pd.DataFrame)->int:
  type_count=df.code_type_local.value_counts()
  if 1 in type_count:
    #Maison
    if type_count[1]==1:
      return df.index.get_loc(df.index[df.code_type_local==1][0])
    else:
      return np.argmax(df.surface_reelle_bati)
  elif 2 in type_count:
    # Appartement
    if type_count[2]==1:
      return df.index.get_loc(df.index[df.code_type_local==2][0])
    else:
      return np.argmax(df.surface_reelle_bati)
  return np.argmax(df.surface_reelle_bati)


def fusion_data(df:pd.DataFrame)->pd.Series:
  iprinc=reduce_get_id(df)
  mut_princ=df.iloc[iprinc]
  values=[]
  # id_mutation
  values.append(mut_princ.id_mutation)
  date=mut_princ.date_mutation.split('-')
  # jour_mutation
  values.append(date[2])
  # mois_mutation
  values.append(date[1])
  # annee_mutation
  values.append(date[0])
  # nature_mutation
  values.append(mut_princ.nature_mutation)
  # valeur_fonciere
  values.append(mut_princ.valeur_fonciere)
  # adresse_numero
  values.append(mut_princ.adresse_numero)
  # adresse_suffixe
  values.append(mut_princ.adresse_suffixe)
  # adresse_nom_voie
  values.append(mut_princ.adresse_nom_voie)
  # adresse_code_voie
  values.append(mut_princ.adresse_code_voie)
  # code_postal
  values.append(mut_princ.code_postal)
  # nom_commune
  values.append(mut_princ.nom_commune)
  # code_departement
  values.append(mut_princ.code_departement)
  # id_parcelle
  values.append(mut_princ.id_parcelle)
  # surface_carrez_total
  cols_carrez=['lot1_surface_carrez','lot2_surface_carrez','lot3_surface_carrez','lot4_surface_carrez','lot5_surface_carrez']
  values.append(df[cols_carrez].sum().sum())
  # surface_reelle_bati_total
  values.append(df.surface_reelle_bati.sum())
  # surface_terrain_total
  values.append(df.surface_terrain.sum())
  # nombre_lots
  values.append(df.nombre_lots.sum())
  # nombre_maisons
  # surface_carrez_maisons
  # surface_reelle_bati_maisons
  # surface_terrain_maisons
  def surface_divide(indice:int):
    # nombre_local
    type_locaux=df.code_type_local.value_counts()
    values.append(0 if indice not in type_locaux else type_locaux[indice])
    # surface_carrez_local
    values.append(df[df.code_type_local==indice][cols_carrez].sum().sum())
    # surface_reelle_bati_local
    values.append(df[df.code_type_local==indice].surface_reelle_bati.sum().sum())
    # surface_terrain_local
    values.append(df[df.code_type_local==indice].surface_terrain.sum().sum())
  surface_divide(1)
  # nombre_appartements
  # surface_carrez_appartements
  # surface_reelle_bati_appartements
  # surface_terrain_appartements
  surface_divide(2)
  # nombre_dependences
  # surface_carrez_dependences
  # surface_reelle_bati_dependences
  # surface_terrain_dependences
  surface_divide(3)
  # nombre_pieces_principales
  values.append(df.nombre_pieces_principales.sum())
  # nature_culture
  values.append(mut_princ.nature_culture)
  # nature_culture_speciale
  values.append(mut_princ.nature_culture_speciale)
  # longitude
  values.append(mut_princ.longitude)
  # latitude
  values.append(mut_princ.latitude)
  return pd.Series({ k:v for k,v in zip(alt_df_cols,values)}, index=alt_df_cols)

def applyParallel(dfGrouped, func):
    retLst = Parallel(n_jobs=multiprocessing.cpu_count())(delayed(func)(group) for _, group in dfGrouped)
    return pd.DataFrame(retLst,columns=alt_df_cols)


# Traitements des fichiers

In [8]:
for annee in tqdm(range(2022,2023)):
  df=pd.read_csv(f"original/{annee}.csv",encoding='utf8')
  df=clean_df(df)
  print(annee)
  new_df=applyParallel(df.groupby("id_mutation"), fusion_data)
  # new_df=df.groupby("id_mutation").apply(fusion_data)
  new_df=new_df.set_index("id_mutation")
  new_df.to_pickle(f'clean_data/{annee}_cleaned.pkl.bz2',compression="bz2")

  0%|          | 0/1 [00:00<?, ?it/s]

2022


100%|██████████| 1/1 [08:35<00:00, 515.83s/it]


In [4]:
p = Path("original")
# csv to parquet
for fp in tqdm(p.glob("*.csv")):
  df=pd.read_csv(fp,encoding='utf8')
  df.iloc[:,[10,12,14,16,17,18,20,22,24,26,35,36]] = df.iloc[:,[10,12,14,16,17,18,20,22,24,26,35,36]].astype('str')
  df.to_pickle(str(fp.with_suffix(".pkl"))+'.bz2',compression="bz2")

0it [00:00, ?it/s]c:\Users\Romain\Programmation\DVF\.venv\Lib\site-packages\IPython\core\interactiveshell.py:3508: DtypeWarning: Columns (10,12,14,16,17,18,20,22,35,36) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
1it [01:13, 73.33s/it]c:\Users\Romain\Programmation\DVF\.venv\Lib\site-packages\IPython\core\interactiveshell.py:3508: DtypeWarning: Columns (10,12,14,16,17,18,20,22,24,35,36) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
2it [02:45, 84.69s/it]c:\Users\Romain\Programmation\DVF\.venv\Lib\site-packages\IPython\core\interactiveshell.py:3508: DtypeWarning: Columns (10,12,14,16,17,18,20,22,24,26,35,36) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
4it [05:50, 90.72s/it]c:\Users\Romain\Programmation\DVF\.venv\Lib\site-packages\IPython\core\interact

In [12]:
p = Path("original/")
# csv to parquet
for fp in tqdm(list(p.glob("*.bz2"))):
  df=pd.read_pickle(fp,compression="bz2")
  df.to_pickle(fp,compression="bz2",protocol=4)

100%|██████████| 7/7 [20:30<00:00, 175.82s/it]


In [10]:
df

,id_mutation,jour_mutation,mois_mutation,annee_mutation,nature_mutation,valeur_fonciere,adresse_numero,adresse_suffixe,adresse_nom_voie,adresse_code_voie,...,surface_terrain_appartements,nombre_dependances,surface_carrez_dependances,surface_reelle_bati_dependances,surface_terrain_dependances,nombre_pieces_principales,nature_culture,nature_culture_speciale,longitude,latitude
0,2022-1,3,1,2022,Vente,55000.0,13.0,<EMPTY>,RUE DE LA LIBERTE,2280,...,0.0,0,0.0,0.0,0.0,1.0,<EMPTY>,<EMPTY>,5.218706,46.198054
1,2022-10,10,1,2022,Vente,580000.0,45.0,<EMPTY>,RUE DES COMBES,0119,...,0.0,5,0.0,0.0,2386.0,6.0,sols,<EMPTY>,5.343466,45.995132
2,2022-100,4,1,2022,Vente,588140.0,100.0,<EMPTY>,CHE DE LA PETITE PRAIRIE,0191,...,0.0,1,0.0,0.0,4.0,5.0,sols,<EMPTY>,6.043660,46.261303
3,2022-1000,21,1,2022,Vente,85000.0,0.0,<EMPTY>,VERNANS,B366,...,0.0,2,0.0,0.0,157.0,0.0,jardins,<EMPTY>,5.532992,45.748300
4,2022-10000,7,3,2022,Vente,46000.0,1.0,<EMPTY>,PL JEAN JAURES,2140,...,0.0,1,0.0,0.0,0.0,3.0,<EMPTY>,<EMPTY>,2.607086,46.339018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
508832,2022-99995,4,1,2022,Vente,140000.0,63.0,<EMPTY>,RUE DE LA MARNE,0700,...,0.0,1,0.0,0.0,567.0,4.0,sols,<EMPTY>,-4.316033,48.569072
508833,2022-99996,13,1,2022,Vente,70000.0,0.0,<EMPTY>,KERVRON,B082,...,0.0,2,0.0,0.0,1195.0,0.0,terrains a bâtir,<EMPTY>,-3.775067,48.686216
508834,2022-99997,17,1,2022,Vente,1500.0,0.0,<EMPTY>,FAVIC,B017,...,0.0,1,0.0,0.0,121.0,0.0,terres,<EMPTY>,-4.029762,48.705180
508835,2022-99998,20,1,2022,Vente,538045.0,5813.0,<EMPTY>,KERGLAS,B104,...,0.0,8,0.0,0.0,17143.0,12.0,terres,<EMPTY>,-3.727567,48.482225
